using an example from: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm
from collections import Counter
import en_core_web_sm


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maggi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

C:\Users\maggi\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
# spacy for lemmatization
import spacy
from spacy import displacy

In [4]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# Logging
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


C:\Users\maggi\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [72]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [73]:
len(stop_words)

179

In [74]:
more_stop_words = ['mr','reply','say','go','see','know','get','look','do','take','tell','want','come',\
                  'feel','even','stand','turn','speak','yet','ever','word','back','day','live','rather',\
                  'miss','wish','bring','say','call','do','give','send','set','insist','several','next',\
                  'put','perhaps','thus','much','day','many','declare','other','show','sort','hold',\
                   'always','begin','meet','exclaim','says','said','goes','sees','knows','gets','looks'\
                  'does','takes','tells','wants','seen','seeing','saying']
stop_words.extend(more_stop_words)

In [27]:
#All Gutenberg metadata
metadata = pd.read_csv('../data/metadata/metadata.csv')

In [28]:
#Select detective/mystery fiction and tales in English
kw = ['fiction','tale']
subj = ['detective', 'mystery','mysteries']
metadata = metadata[metadata['language']=='[\'en\']']
metadata = metadata[metadata['subjects'].apply(lambda x: any([k in x.lower() for k in kw]))]
metadata = metadata[metadata['subjects'].apply(lambda x: any([k in x.lower() for k in subj]))]

#Ids to list
ids = list(metadata['id'])

#List the ids and path for these titles
txts = [[id, '../data/text\\' + id + '_text.txt'] for id in ids]

#note any files that don't exist (duplicates or files that were not utf8) and remove them from the metadata df
docs = []

for idx, filename in tqdm(txts):
    try:
        with open(filename, 'r', encoding = 'utf8') as txt:
            # added encoding = 'utf8' to prevent a continuation byte error
            #txt = f
            #pdf = pdftotext.PDF(f)
            txt = '\n\n'.join(txt)#.lower()
            txt = re.sub('-\s+', '', txt)
            txt = re.sub('--',' ', txt)
            txt = re.sub("\'", "", txt)
            txt = ' '.join(re.findall('[a-zA-z\-]{2,}', txt))
            txt = txt[0:50000]
            docs.append(txt)
    except FileNotFoundError:
        print('WARNING: file not found ', filename)
        metadata = metadata[metadata['id']!=idx]

#Does everything match?
print(len(metadata) == len(docs))
print(len(metadata))

  0%|          | 0/1093 [00:00<?, ?it/s]

True
1052


In [29]:
docs[0:2]

['Proofreaders THE SECRET OF THE TOWER BY ANTHONY HOPE AUTHOR OF THE PRISONER OF ZENDA RUPERT OF HENTZAU ETC CONTENTS DOCTOR MARYS PAYING GUEST II THE GENERAL REMEMBERS III MR SAFFRON AT HOME IV PROFESSIONAL ETIQUETTE FAMILIAR IMPLEMENT VI ODD STORY OF CAPTAIN DUGGLE VII GENTLEMANLY STRANGER VIII CAPTAIN ALEC RAISES HIS VOICE IX DOCTOR MARYS ULTIMATUM THAT MAGICAL WORD MOROCCO XI THE CAR BEHIND THE TREES XII THE SECRET OF THE TOWER XIII RIGHT OF CONQUEST XIV THE SCEPTER IN THE GRAVE XV NORMAL CASE XVI DEAD MAJESTY XVII THE CHIEF MOURNERS XVIII THE GOLD AND THE TREASURE CHAPTER DOCTOR MARYS PAYING GUEST Just in time wasnt it asked Mary Arkroyd Two days before the the ceremony Mercifully it had all been kept very quiet because it was only three months since poor Gilly was killed forget whether you ever met Gilly My half-brother you know Only once in Collingham Gardens He had an _exeat_ and dashed in one Saturday morning when we were just finishing our work Dont you remember Yes think do 

In [10]:
#docs = docs[:5]

In [11]:
#nlp = en_core_web_sm.load()

In [12]:
#for idx, doc in tqdm(enumerate(docs)):
#    this_doc = nlp(doc)
#    ents = [e.text for e in this_doc.ents]
#    for item in ents:
#        doc = doc.replace(' ' + item + ' ', ' ')
#    docs[idx] = doc

0it [00:00, ?it/s]

In [30]:
#docs[0]

In [47]:
def sent_to_words(sentences):
    for sentence in tqdm(sentences):
        tagged_words = nltk.tag.pos_tag(sentence.split())
        no_names = [word for word, tag in tagged_words if tag != 'NNP' and tag != 'NNPS'] # remove proper nouns
        yield(gensim.utils.simple_preprocess(str(no_names), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(docs))

#print(data_words[:1])

  0%|          | 0/1052 [00:00<?, ?it/s]

In [69]:
#print(data_words[0])

In [49]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

['proofreaders', 'the', 'of', 'of', 'of', 'in', 'time', 'wasnt', 'it', 'asked', 'two', 'days', 'before', 'the', 'the', 'ceremony', 'it', 'had', 'all', 'been', 'kept', 'very', 'quiet', 'because', 'it', 'was', 'only', 'three', 'months', 'since', 'poor', 'was', 'killed', 'forget', 'whether', 'you', 'ever', 'met', 'half', 'brother', 'you', 'know', 'only', 'once', 'in', 'he', 'had', 'an', 'and', 'dashed', 'in', 'one', 'morning', 'when', 'we', 'were', 'just', 'finishing', 'our', 'work', 'you', 'remember', 'yes', 'think', 'do', 'but', 'since', 'my', 'engagement', 'gone', 'into', 'colors', 'of', 'course', 'gone', 'back', 'into', 'mourning', 'now', 'and', 'everything', 'was', 'ready', 'settlements', 'and', 'so', 'on', 'you', 'know', 'and', 'rooms', 'taken', 'at', 'and', 'then', 'it', 'all', 'came', 'out', 'mean', 'think', 'he', 'really', 'must', 'have', 'had', 'shell', 'shock', 'as', 'he', 'said', 'even', 'though', 'the', 'doctor', 'seemed', 'to', 'doubt', 'it', 'he', 'gave', 'the', 'as', 'refe

In [50]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [75]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
#nlp = spacy.load('en', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])

In [76]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

In [77]:
#id2word[871]

In [78]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('absolute', 1),
  ('accent', 1),
  ('accept', 1),
  ('accessible', 1),
  ('accustomed', 1),
  ('ache', 1),
  ('acquaint', 1),
  ('acre', 1),
  ('act', 1),
  ('active', 1),
  ('activity', 1),
  ('actually', 1),
  ('add', 3),
  ('address', 1),
  ('admiration', 1),
  ('admit', 1),
  ('advice', 1),
  ('affect', 1),
  ('afford', 1),
  ('afraid', 2),
  ('afternoon', 2),
  ('agent', 1),
  ('aggressive', 1),
  ('agreeable', 1),
  ('air', 1),
  ('almost', 3),
  ('alone', 1),
  ('also', 2),
  ('ambitious', 1),
  ('amenity', 1),
  ('amused', 1),
  ('angry', 1),
  ('answer', 1),
  ('antiquity', 1),
  ('anyhow', 4),
  ('apparently', 1),
  ('appear', 1),
  ('approach', 1),
  ('ardent', 1),
  ('arm', 2),
  ('armistice', 2),
  ('arrangement', 1),
  ('arrival', 1),
  ('ask', 4),
  ('aspect', 1),
  ('assign', 1),
  ('assist', 1),
  ('astonish', 1),
  ('attentive', 1),
  ('attractive', 1),
  ('audience', 1),
  ('avenue', 2),
  ('away', 1),
  ('awfully', 1),
  ('bad', 2),
  ('bag', 1),
  ('bank', 1),
 

In [79]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=500,
                                           passes=20,
                                           iterations = 400,
                                           alpha='auto',
                                           per_word_topics=True)
#num_topics = 6
#chunksize = 2000
#passes = 20
#iterations = 400
#eval_every = None  # Don't evaluate model perplexity, takes too much time.

In [80]:
dir(lda_model)
lda_model.save('../model_artifacts/model.lda02')

In [81]:
model = gensim.models.ldamodel.LdaModel.load('../model_artifacts/model.lda02')

In [82]:
model.print_topics(num_words = 100)

[(0,
  '0.008*"go" + 0.008*"time" + 0.007*"man" + 0.006*"do" + 0.006*"make" + 0.005*"think" + 0.005*"good" + 0.005*"old" + 0.005*"find" + 0.005*"little" + 0.005*"father" + 0.004*"place" + 0.004*"case" + 0.004*"thing" + 0.004*"well" + 0.004*"come" + 0.004*"take" + 0.004*"year" + 0.004*"leave" + 0.004*"look" + 0.004*"name" + 0.004*"ask" + 0.004*"boy" + 0.004*"detective" + 0.004*"horse" + 0.003*"brother" + 0.003*"right" + 0.003*"girl" + 0.003*"home" + 0.003*"use" + 0.003*"first" + 0.003*"know" + 0.003*"away" + 0.003*"town" + 0.003*"story" + 0.003*"money" + 0.003*"get" + 0.003*"fact" + 0.003*"hand" + 0.003*"young" + 0.003*"mind" + 0.003*"business" + 0.003*"stable" + 0.003*"matter" + 0.003*"way" + 0.003*"run" + 0.003*"box" + 0.003*"give" + 0.003*"interest" + 0.003*"also" + 0.003*"course" + 0.002*"work" + 0.002*"lad" + 0.002*"carry" + 0.002*"great" + 0.002*"see" + 0.002*"night" + 0.002*"professor" + 0.002*"whole" + 0.002*"scout" + 0.002*"s" + 0.002*"talk" + 0.002*"present" + 0.002*"call" + 0

In [85]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"go" + 0.008*"time" + 0.007*"man" + 0.006*"do" + 0.006*"make" + '
  '0.005*"think" + 0.005*"good" + 0.005*"old" + 0.005*"find" + 0.005*"little"'),
 (1,
  '0.001*"sheepskin" + 0.000*"iceboat" + 0.000*"catwalk" + 0.000*"billowing" + '
  '0.000*"unshoveled" + 0.000*"dispirited" + 0.000*"riverview" + '
  '0.000*"unscrew" + 0.000*"motherhood" + 0.000*"parka"'),
 (2,
  '0.004*"yust" + 0.003*"der" + 0.002*"tiamont" + 0.002*"pishness" + '
  '0.002*"haf" + 0.001*"plait" + 0.001*"is" + 0.001*"carat" + 0.001*"aix" + '
  '0.001*"bain"'),
 (3,
  '0.009*"go" + 0.008*"boy" + 0.008*"make" + 0.006*"time" + 0.006*"get" + '
  '0.006*"come" + 0.005*"girl" + 0.005*"think" + 0.005*"way" + 0.005*"right"'),
 (4,
  '0.011*"man" + 0.005*"make" + 0.005*"time" + 0.004*"life" + 0.004*"find" + '
  '0.004*"great" + 0.004*"good" + 0.004*"well" + 0.004*"seem" + 0.003*"first"'),
 (5,
  '0.006*"old" + 0.006*"little" + 0.006*"young" + 0.006*"man" + 0.005*"house" '
  '+ 0.005*"girl" + 0.005*"eye" + 0.005*"lo

In [83]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.238789256160985

Coherence Score:  0.297852096032393


In [84]:
# Visualize the topics
pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis = gensimvis.prepare(model, corpus, id2word, mds = 'mmds')
#vis = gensimvis.prepare(model, corpus, dictionary, mds = 'mmds', sort_topics=False)
pyLDAvis.save_html(vis, 'lda03.html')
#vis

We preprocess the novels by lemmatizing the words, removing
punctuation, function words and names, and splitting the remaining text in chunks of 1000
tokens. We use MALLET to create a topic model with 50 topics. Fig. 1 shows an overview of the
topics with their proportion across the corpus.

Next steps: 

identify and remove named entities before tokenizing

use entire texts instead of first x characters

break into chunks of 1000 tokens for topic modeling -- how does this actually work if you are processing multiple books/texts?

iterate through models with various parameters